<a href="https://colab.research.google.com/github/pethodoma/BME-DeepLearning-BirdCLEF_2023/blob/main/final_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import shutil
from tqdm.auto import tqdm

In [ ]:
!wget -q -O training_files.zip https://www.dropbox.com/scl/fi/3il8m9feo2piqnsodwguh/atleast_50_png.zip?rlkey=9zipwrq01xv9r29n9xgcci6q8&dl=0
!unzip -q 'training_files.zip'

In [ ]:
def read_file_paths(main_directory):
    main_directory = main_directory
    file_paths = []

    # go through all folders and get the paths of all .ogg audio files
    for root, directories, files in os.walk(main_directory):
        for file in files:
            if file.endswith('.png'):
                file_path = os.path.join(root, file)
                file_paths.append(file_path)

    # os.walk may not go in alphabetical order thus it needs to be sorted
    file_paths.sort()
    return file_paths

In [ ]:
# Creating the dict with the numerical categories to each filepath
df = pd.read_csv('atleast_50.csv')
df['Category'] = range(len(df))
# Creating the dictionary that maps the labels to the filepaths
labelmapper = {filename: df[df['Bird'] == filename.split(os.path.sep)[-2]]['Category'].values[0] for filename in read_file_paths('spectrograms')}
# Splitting the data
filepaths = list(labelmapper.keys())
labels = list(labelmapper.values())
train_paths, test_valid_paths, train_labels,  test_valid_labels = train_test_split(filepaths, labels, test_size=0.2, random_state=42, stratify=labels)
test_paths, valid_paths, _, _ = train_test_split(test_valid_paths, test_valid_labels, test_size=0.5, random_state=42, stratify=test_valid_labels)
# Create directories for training and validation data
os.makedirs('train', exist_ok=True)
os.makedirs('test', exist_ok=True)
os.makedirs('validation', exist_ok=True)
# Creating the subdirectories with the bird names
for bird in df['Bird']:
    os.makedirs(os.path.join('train', bird), exist_ok=True)
    os.makedirs(os.path.join('test', bird), exist_ok=True)
    os.makedirs(os.path.join('validation', bird), exist_ok=True)
# Moving the files to the train and validation folders
for filepath in train_paths:
    shutil.move(filepath, os.path.join('train', filepath.split(os.path.sep)[-2], filepath.split(os.path.sep)[-1]))
for filepath in test_paths:
    shutil.move(filepath, os.path.join('test', filepath.split(os.path.sep)[-2], filepath.split(os.path.sep)[-1]))
for filepath in valid_paths:
    shutil.move(filepath, os.path.join('validation', filepath.split(os.path.sep)[-2], filepath.split(os.path.sep)[-1]))
# Deleting the original directory
shutil.rmtree('spectrograms')

In [ ]:
batch_size = 128
# Create an ImageDataGenerator for training data
# Datagenerator for training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(128, 312),
    batch_size=batch_size,
    class_mode='categorical'
)
# Datagenerator for validation
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(
    'test',
    target_size=(128, 312),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 66200 images belonging to 192 classes.
Found 8275 images belonging to 192 classes.


In [ ]:
# Get class indices from the generator
class_indices = train_generator.class_indices
class_labels = list(class_indices.values())
# Calculate class weights
class_weights = class_weight.compute_class_weight('balanced', classes=class_labels, y=train_generator.classes)
# Map class weights
class_weights_dict = {}
for i, class_label in enumerate(class_labels):
    class_weights_dict[class_label] = class_weights[i]

In [ ]:
# Defining callbacks
es = tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss', verbose=1)
mc = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True, verbose=1)

In [ ]:
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.5 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from kerastuner.tuners import RandomSearch

<ipython-input-9-511beda95521>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
num_classes = 192

In [ ]:
# Function to help hyperparameter optimization
def build_model(hp):
    model = Sequential()

    model.add(Conv2D(hp.Int('conv1_units', min_value=16, max_value=64, step=8), (5, 5), activation='relu', input_shape=(128, 312, 3)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(hp.Int('conv2_units', min_value=8, max_value=32, step=8), (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(hp.Int('dense_units', min_value=64, max_value=256, step=32), activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy', 'categorical_crossentropy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model

In [ ]:
# Using RandomSearch and running 5 trials, with each trial having 5 epochs
num_epochs = 5

# Configuring the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='keras_tuner_logs',
)

# Searching the best parameters
tuner.search(train_generator,
             epochs=num_epochs,
             validation_data=valid_generator,
             callbacks=[es, mc],
             class_weight=class_weights_dict,
             workers=6,
             use_multiprocessing=True)

# The best found hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Building the final model
final_model = tuner.hypermodel.build(best_hps)

Trial 5 Complete [00h 32m 14s]
val_accuracy: 0.29631420969963074

Best val_accuracy So Far: 0.46308156847953796
Total elapsed time: 02h 33m 10s


In [ ]:
# The best found parameters were good enough for 0.46 validation accuracy after 5 epochs
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 124, 308, 32)      2432      
                                                                 
 dropout_2 (Dropout)         (None, 124, 308, 32)      0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 62, 154, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 58, 150, 16)       12816     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 29, 75, 16)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 34800)            

In [ ]:
num_epochs = 10

In [ ]:
# Training the model with the best found parameters, for 10 epochs
final_model.fit(train_generator,
                epochs=num_epochs,
                validation_data=valid_generator,
                class_weight=class_weights_dict,
                workers=6,
                use_multiprocessing=True)

# Saving the final model
# We could reach almost .53 validation accuracy with this model
final_model.save('final_model.h5')

Epoch 1/10
518/518 [==============================] - 347s 666ms/step - loss: 4.1554 - accuracy: 0.1466 - categorical_crossentropy: 4.2170 - precision_1: 0.7309 - recall_1: 0.0415 - val_loss: 3.4099 - val_accuracy: 0.2830 - val_categorical_crossentropy: 3.4099 - val_precision_1: 0.8438 - val_recall_1: 0.0777
Epoch 2/10
518/518 [==============================] - 353s 675ms/step - loss: 2.4849 - accuracy: 0.3606 - categorical_crossentropy: 2.9259 - precision_1: 0.7614 - recall_1: 0.1935 - val_loss: 2.9837 - val_accuracy: 0.3403 - val_categorical_crossentropy: 2.9837 - val_precision_1: 0.7398 - val_recall_1: 0.1594
Epoch 3/10
518/518 [==============================] - 361s 690ms/step - loss: 1.3893 - accuracy: 0.5123 - categorical_crossentropy: 2.1540 - precision_1: 0.7905 - recall_1: 0.3757 - val_loss: 2.6672 - val_accuracy: 0.4022 - val_categorical_crossentropy: 2.6672 - val_precision_1: 0.7432 - val_recall_1: 0.2231
Epoch 4/10
518/518 [==============================] - 360s 692ms/step 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
